In [56]:
class CarInfo:
    def __init__(self, id, model_name: str, body_type: str, fuel_type: str, price: int, power: int,
                 fuel_efficiency: str, model_year: int, size: int, engine_type: str, image_link: str, brand: str):
        self.__id = id
        self.__model_name = model_name
        self.__body_type = body_type
        self.__fuel_type = fuel_type
        self.__price = price
        self.__power = power
        self.__fuel_efficiency = fuel_efficiency  # 할당 누락 수정
        self.__model_year = model_year
        self.__size = size
        self.__engine_type = engine_type
        self.__image_link = image_link
        self.__brand = brand

    @property
    def id(self):
        return self.__id

    @property
    def model_name(self):
        return self.__model_name

    @property
    def body_type(self):
        return self.__body_type

    @property
    def fuel_type(self):
        return self.__fuel_type

    @property
    def price(self):
        return self.__price

    @property
    def power(self):
        return self.__power

    @property
    def fuel_efficiency(self):
        return self.__fuel_efficiency

    @property
    def model_year(self):
        return self.__model_year

    @property
    def size(self):
        return self.__size

    @property
    def engine_type(self):
        return self.__engine_type

    @property
    def image_link(self):
        return self.__image_link

    @property
    def brand(self):
        return self.__brand

    def __repr__(self):
        return (f'CarInfo({self.__id}, {self.__model_name}, {self.__body_type}, {self.__fuel_type}, {self.__price}, '
                f'{self.__power}, {self.__fuel_efficiency}, {self.__model_year}, {self.__size}, '
                f'{self.__engine_type}, {self.__image_link}, {self.__brand})')

# -------------------------------------------------------


페이지 3에서 오류 발생: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="main_pack"]/div[3]/div[2]/div[1]/div/div[3]/div[3]"}
  (Session info: chrome=135.0.7049.115); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x0000000101246a54 cxxbridge1$str$ptr + 2803960
1   chromedriver                        0x000000010123ecf0 cxxbridge1$str$ptr + 2771860
2   chromedriver                        0x0000000100d8a864 cxxbridge1$string$len + 93028
3   chromedriver                        0x0000000100dd1410 cxxbridge1$string$len + 382736
4   chromedriver                        0x0000000100e12480 cxxbridge1$string$len + 649088
5   chromedriver                        0x0000000100dc57ec cxxbridge1$string$len + 334572
6   chromedriver                        0x000000010120bccc cxxbridge1$str$ptr + 2562928
7   chromedriver    

In [154]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup as bs
import time
import re
from dataclasses import dataclass

# ✅ CarInfo 클래스 정의
@dataclass
class CarInfo:
    id: int
    model_name: str
    body_type: str
    fuel_type: str
    price: int
    power: int
    fuel_efficiency: float
    model_year: str
    size: float
    engine_type: str
    image_link: str
    brand: str

# ✅ 드라이버 실행
driver = webdriver.Chrome()

# ✅ 웹페이지 접속
driver.get("https://naver.com")
time.sleep(3)

# ✅ 검색어 입력
search_box = driver.find_element(By.ID, "query")
search_box.send_keys("네이버 자동차")
search_box.send_keys(Keys.ENTER)
time.sleep(3)

# ✅ 5천만원 이하 필터링
button = driver.find_element(By.XPATH, '//*[@id="main_pack"]/div[3]/div[2]/div[1]/div/div[1]/div/div/div/ul/li[5]/div/a')
button.click()
time.sleep(2)

slider = driver.find_element(By.XPATH, '//*[@id="main_pack"]/div[3]/div[2]/div[1]/div/div[1]/div/div/div/div/div/div/div[2]/div[1]/div[3]/div/span/span[7]')
slider.click()
time.sleep(2)

actions = ActionChains(driver)
for _ in range(2):
    actions.send_keys(Keys.ARROW_LEFT).perform()
time.sleep(2)

apply = driver.find_element(By.XPATH, '//*[@id="main_pack"]/div[3]/div[2]/div[1]/div/div[1]/div/div/div/div/div/div/div[2]/div[2]/a')
apply.click()
time.sleep(2)

# ✅ 상세페이지 URL 수집
url_list = []

for page in range(1, 16):
    try:
        container = driver.find_element(By.XPATH, f'//*[@id="main_pack"]/div[3]/div[2]/div[1]/div/div[3]/div[{page}]')
        car_list = container.find_elements(By.CSS_SELECTOR, ".info_box")

        for car_tag in car_list:
            link = car_tag.find_element(By.CSS_SELECTOR, "a:first-of-type")
            href = link.get_attribute('href')
            if href and href not in url_list:
                url_list.append(href + '%20%EC%A0%95%EB%B3%B4')

        next_button = driver.find_element(By.XPATH, '//*[@id="main_pack"]/div[3]/div[2]/div[1]/div/div[4]/div/a[2]')
        next_button.click()
        time.sleep(3)

    except Exception as e:
        print(f"오류 발생 (페이지 {page}): {e}")
        break  # 다음 페이지 못 찾으면 종료

print(f"✅ 총 수집된 링크 수: {len(url_list)}개")

# ✅ url 순회하며 상세 정보 추출
car_info_list = []

def get_text_from_dd(info_groups, index):
    """ info_group 내부의 특정 dd 값을 가져오는 함수 """
    if len(info_groups) > index:
        dd_tag = info_groups[index].select_one('dd')
        return dd_tag.text.strip() if dd_tag else None
    return None

def extract_first_number(text, return_float=False):
    """ 숫자만 추출해서 반환하는 함수 """
    if not text:
        return None
    numbers = re.findall(r'\d+(?:\.\d+)?', text.replace(',', ''))
    if numbers:
        num = float(numbers[0])
        return round(num, 1) if return_float else int(num)
    return None

for idx, url in enumerate(url_list, 1):
    try:
        driver.get(url)

        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'span.area_text_title strong._text'))
        )

        soup = bs(driver.page_source, 'html.parser')

        # ✅ 모델명
        model_name_tag = soup.select_one('span.area_text_title strong._text')
        model_name = model_name_tag.text.strip() if model_name_tag else None

        # ✅ body_type & model_year
        sub_title_tags = soup.select('div.sub_title .txt')
        body_type = sub_title_tags[0].text.strip() if len(sub_title_tags) > 0 else None
        model_year = sub_title_tags[1].text.strip() if len(sub_title_tags) > 1 else None

        # ✅ image_link
        image_tag = soup.select_one('div.detail_info a.thumb.type_87_87 img')
        image_link = image_tag['src'] if image_tag else None

        # ✅ info_group 내부 데이터
        info_groups = soup.select('dl.info .info_group')

        price_text = get_text_from_dd(info_groups, 0)
        price = extract_first_number(price_text)

        fuel_type = get_text_from_dd(info_groups, 1)

        fuel_efficiency_text = get_text_from_dd(info_groups, 2)
        fuel_efficiency = extract_first_number(fuel_efficiency_text, return_float=True)

        power_text = get_text_from_dd(info_groups, 3)
        power = extract_first_number(power_text)

        engine_type = None
        if len(info_groups) > 6:
            engine_dd = info_groups[6].select_one('dd')
            engine_span = info_groups[6].select_one('span.value_text')
            engine_type = (engine_dd.text.strip() if engine_dd else '') + " " + (engine_span.text.strip() if engine_span else '')

        size = None
        size_text1 = get_text_from_dd(info_groups, 9)
        size_text2 = get_text_from_dd(info_groups, 12)
        if size_text1 and size_text2:
            size_num1 = extract_first_number(size_text1)
            size_num2 = extract_first_number(size_text2)
            if size_num1 and size_num2:
                size = round((size_num2 / size_num1) * 100, 2)

        # ✅ model_name에서 브랜드 추출
        brand = model_name.split()[0] if model_name else None

        # ✅ CarInfo 저장
        if model_name:
            car_info = CarInfo(
                id=idx,
                model_name=model_name,
                body_type=body_type,
                fuel_type=fuel_type,
                price=price,
                power=power,
                fuel_efficiency=fuel_efficiency,
                model_year=model_year,
                size=size,
                engine_type=engine_type,
                image_link=image_link,
                brand=brand
            )
            car_info_list.append(car_info)
            print(f"[{idx}] {car_info}")
        else:
            print(f"[{idx}] 모델명 없음")

    except TimeoutException:
        print(f"[{idx}] 타임아웃 발생: {url}")
    except Exception as e:
        print(f"[{idx}] 에러 발생: {e} URL: {url}")

print("\n✅ 최종 수집된 CarInfo 수:", len(car_info_list))

# ✅ 드라이버 종료
driver.quit()


오류 발생 (페이지 15): Message: element click intercepted: Element <a nocr="" onclick="goOtherTCR(this, 'a=nco_x58*3.page&amp;r=1&amp;i=18000090_000000000000');return false;" href="#" class="pg_next" data-kgs-page-action-next="" aria-disabled="true">...</a> is not clickable at point (474, 486). Other element would receive the click: <div class="pgs">...</div>
  (Session info: chrome=135.0.7049.115)
Stacktrace:
0   chromedriver                        0x0000000100f9ea54 cxxbridge1$str$ptr + 2803960
1   chromedriver                        0x0000000100f96cf0 cxxbridge1$str$ptr + 2771860
2   chromedriver                        0x0000000100ae2864 cxxbridge1$string$len + 93028
3   chromedriver                        0x0000000100b2ef3c cxxbridge1$string$len + 406076
4   chromedriver                        0x0000000100b2d4a8 cxxbridge1$string$len + 399272
5   chromedriver                        0x0000000100b2b2bc cxxbridge1$string$len + 390588
6   chromedriver                        0x0000000100b2a6b8

In [156]:
import mysql.connector

conn = mysql.connector.connect(
    host='localhost',
    port=3306, # 기본포트 3306인 경우 생략 가능
    user='skn14',
    password='skn14',
    database='teamdb'
)

cursor = conn.cursor()

# 커서를 통한 쿼리 수행
cursor.execute('select * from car_info')
rows = cursor.fetchall() # 결과집합 반환
for row in rows:
    print(row)

cursor.close()
conn.close()